In [1]:
import numpy as np
from utils.train import train_wrapper, final_train
import cv2
import os

Segmentation Models: using `keras` framework.


# Model Training
Code to train a U-net with ResNet34 backbone on the preprocessed images and masks. The training call below contains hyperparameter values that have been optimized during initial experiments. To test different hyperparameters, you can use the ```train_wrapper``` function, which will run training with k-crossfold validation.

Hyperparameters:
- augmentation technique: flipping, rotation, cropping, brightness contrast changes, sharpen blur changes, Gaussian noise injection
- augmentation design: on-the-fly (real time transformation) or offline (fixed before training)
- augmentation factor (for offline augmentation only): controls the size of the augmented dataset
- patch size (```im_size```): 32, 64, 128, 256, or 480
- loss function: categorical cross-entropy loss or categorical focal dice loss
- dropout (use_dropout): uses dropout layers after each upsampling operation in the decoder with a probability of 0.5. The dropout probability can be changed in ```models/segmentation_models_quebvel/segmentation_models/models/unet.py```
- batchnorm (use_batchnorm): uses batchnorm layers in the encoder and decoder after convolution
- fine-tuning: fine-tune (pretrain on ImageNet and set all layers trainable during training), encoder freeze (pretrain on ImageNet and freeze encoder during training), or train from scratch
- class weighting: accounts for class imbalance using weights in the loss function (weights are calculated from the class distributions)
- optimizer: Adam optimizer with learning rate 0.001 (for other optimizers available see https://github.com/qubvel/segmentation_models)
- batch size
- number of training epochs (use ````early_stop``` argument for early stopping)

Augmentation techniques have been preselected to simulate changes in conditions. The augmentation techniques can be combined in different ways. To do so, change the code in ```utils/augmentation.py```. In initial experiments, most of the augmentation techniques did not improve performance, so the current options contain single techniques only.

Patch size means that the model can be trained on smaller patches of the input image. The patches are extracted using a sliding window approach. This results in overlapping patches in the case of 256 patch size, and non-overlapping patches in the other cases. The patch extraction technique can be changed to extract random patches from an image using the ```patch_mode``` argument. When using patch extraction, the prediction function must be adjusted (```prediction_sample.ipynb```).

To decide between pre-training and training from scratch, use the ```train_transfer``` argument. When pre-training, use the ```encoder_freeze``` argument to decide between fine-tuning and encoder freezing.

For the implementation of the categorical focal dice loss function see https://github.com/qubvel/segmentation_models.

To use the training function for different images (not tested yet), you may need to change the image size parameter (im_size).

For monitoring training and validation curves using the Weights & Biases dashboard, you can uncomment the respective code lines in the ```utils/train.py``` function.

A note on performance evaluation:
The test data set currently contains only two images and is unlikely to represent the training data distribution, not to mention the distribution of the real data. Therefore, numerical performance estimates should be considered with caution, and must be regarded in combination with qualitative results (```prediction_sample.ipynb```). Hyperparameter optimization was performed using k-crossfold validation to give a better decision base.

In [3]:
images = np.load('./data/images.npy')
masks = np.load('./data/masks.npy')

test_images = np.load('./data/test_ds_np/480_im.npy')
test_masks = np.load('./data/test_ds_np/480_ma.npy')

In [4]:
_ = final_train(images, masks, test_images, test_masks, loss='focal_dice', weight_classes=True, epochs=100, im_size=480, base_pref='test_gitlab', augmentation='on_fly', mode=4, use_dropout=True, train_transfer='imagenet', batch_size=2)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, 480, 480, 3  0           []                               
                                )]                                                                
                                                                                                  
 bn_data (BatchNormalization)   (None, 480, 480, 3)  9           ['data[0][0]']                   
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 486, 486, 3)  0          ['bn_data[0][0]']                
                                                                                                  
 conv0 (Conv2D)                 (None, 240, 240, 64  9408        ['zero_padding2d[0][0]']   